## Imports

In [7]:
import requests

import os, sys
from tda import auth, client
from tda.orders.equities import equity_buy_market, equity_buy_limit
from tda.orders.common import Duration, Session
import tda

# currentdir = os.path.dirname(os.path.realpath(__file__))
currentdir = os.path.abspath('')
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import config

from selenium import webdriver
import time
import json

token_path = "token"

DRIVER_PATH = "/home/hopper/chromedriver"
driver = webdriver.Chrome(DRIVER_PATH)

redirect_uri = "https://localhost"


PATH = "/home/hopper/chromedriver"
token_path = "token"

try:
    c = auth.client_from_token_file(token_path, config.api_key)
except FileNotFoundError:
    c = auth.client_from_login_flow(driver, config.api_key, redirect_uri, token_path)


## Print Financhill Score of all TDA position

In [8]:
positions = c.Account.Fields.POSITIONS
r = c.get_account(config.tda_acct_num, fields=positions)

stocks = r.json()['securitiesAccount']['positions']

stock_symbols = [] #append later

# for stock, data in my_stocks.items():
for stock in stocks:
    break
    ticker = stock['instrument']['symbol']
    if ticker == 'MMDA1':
        continue
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get("https://financhill.com/search/stock-score/" + ticker)
    time.sleep(2)

    print(ticker)
    score = int(driver.find_element_by_tag_name("h2").text)
    print(score)

    # if (score < 40):
    #     r.order_sell_trailing_stop(stock, data['quantity'], 1)
    #     soldStocks.append(stock)

    driver.quit()

## Buy top stocks from Financhill

TODO: Only buy if I don't own any to encourage diversification

In [9]:
print('1.1')
# All this scraping code works
driver.get("https://financhill.com/screen/stock-score")
time.sleep(2)
driver.find_element_by_css_selector(
    'span[data-sort-name="stock_score_normalized"]'
).click()
time.sleep(2)
tickers = driver.find_elements_by_tag_name("td")
print('1.2')

positions = c.Account.Fields.POSITIONS
r = c.get_account(config.tda_acct_num, fields=positions)
print('1.3')
stocks = r.json()['securitiesAccount']['positions']

stock_symbols = [] #append later

# for stock in stocks:
#     stock_symbols.append(stock['instrument']['symbol'])

new_stocks_found = False

i = 60
print('2')
# [0]:Ticker, [1]:Share Price, [2]:Rating, [3]:Score, [4]:Rating Change Date, [5]:Price Change %
while i < 200:

    # Get ticker and price of stock
    ticker = str(tickers[i].text)
    share_price = float(tickers[i + 1].text)

    # Calculate how many shares to buy in order to equal about $1000
    desired_dollar_amount = 1000 # How many dollars of each stock to buy
    num_shares = round(desired_dollar_amount / share_price)

    if (not(ticker in stock_symbols)):

        # Build and place order
        order = equity_buy_market(ticker, num_shares)
        r = c.place_order(config.tda_acct_num, order)
        print("Bought " + str(num_shares) + " shares of " + ticker)
        new_stocks_found = True

    i += 10

if (not new_stocks_found):
    print("You already own all the top stocks")

driver.quit()

1.1
1.2
1.3


TypeError: list indices must be integers or slices, not str